In [1]:
# dataframe management
import pandas as pd             

# numerical computation
import numpy as np

# visualization library
import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline
#----------------------------------------------------------------------------------------------------
from datetime import date
#---------------------------------------------------------------------------------------------------------
# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")

from bokeh.layouts import gridplot
from bokeh.plotting import figure

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
sales_string_date=pd.read_csv('train.csv')
sales = sales_string_date
#sales["Date"] = pd.to_datetime(sales["Date"], infer_datetime_format=True)
sales

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,Mean_TemperatureC,Mean_VisibilityKm,Mean_Wind_SpeedKm_h,Min_Dew_PointC,Min_Humidity,Min_Sea_Level_PressurehPa,Min_TemperatureC,Min_VisibilitykM,Precipitationmm,WindDirDegrees
0,1000,01/03/2016,0,1,0,Hyper Market,General,326,7,495,...,1,11.0,16,-2,70,1029,1,6.0,0.00,23
1,1000,02/03/2016,0,1,0,Hyper Market,General,326,7,608,...,3,13.0,10,-2,58,1025,1,10.0,0.00,56
2,1000,04/03/2016,0,1,0,Hyper Market,General,326,7,665,...,3,11.0,10,-3,55,1023,2,8.0,0.00,22
3,1000,05/03/2016,0,1,0,Hyper Market,General,326,7,630,...,3,15.0,10,-6,25,1022,-1,10.0,0.00,108
4,1000,06/03/2016,0,0,0,Hyper Market,General,326,7,0,...,1,12.0,5,-6,48,1022,-5,5.0,0.00,46
5,1000,07/03/2016,0,1,1,Hyper Market,General,326,7,763,...,4,22.0,11,-4,26,1016,-2,16.0,0.00,68
6,1000,08/03/2016,0,1,1,Hyper Market,General,326,7,675,...,6,21.0,8,-4,24,1008,-2,18.0,0.00,62
7,1000,09/03/2016,0,1,1,Hyper Market,General,326,7,690,...,7,22.0,10,-2,25,1000,-1,13.0,0.00,53
8,1000,10/03/2016,0,1,1,Hyper Market,General,326,7,617,...,11,14.0,6,0,28,999,6,7.0,0.00,54
9,1000,11/03/2016,0,1,1,Hyper Market,General,326,7,636,...,12,9.0,11,5,37,998,7,6.0,0.51,30


In [12]:
sales_string_date=pd.read_csv('train.csv')
sales = sales_string_date

def AddMonthYearColumn(d):
    d["MonthYear"] = d[["Date"]].applymap(lambda x : "/".join(x.split("/")[1:])) # get the month
    return d

# return only the data in the months and years specified, passed as a list (e.g. ["01/2017","02/2017"])
def FilterMonthsYears(data, monthsYears):
    d = AddMonthYearColumn(data)
    mask =(d.columns=="MonthYear")
    d = d[d["MonthYear"].isin(monthsYears)]
    return d[d.columns[~mask]]

# return all the data except the ones in the months and years specified, passed as a list (e.g. ["01/2017","02/2017"])
def NegativeFilterMonthsYears(data, monthsYears):
    d = AddMonthYearColumn(data)
    mask =(d.columns=="MonthYear")
    d = d[~d["MonthYear"].isin(monthsYears)]
    return d[d.columns[~mask]]

# compute the region's error over the given data
# needed Columns: "NumberOfActualSales" the actual sales, "NumberOfSales" the predicted sales.
def RegionError(region, data):
    d = data[data["Region"] == region][["NumberOfActualSales", "NumberOfSales"]]
    res = abs(d["NumberOfActualSales"]-d["NumberOfSales"]).agg("sum")
    return res / d["NumberOfActualSales"].agg("sum")


## test: should return 0 being NumberOfActualSales = NumberOfSales
sales_month = sales
sales_month["NumberOfActualSales"] = sales_month["NumberOfSales"]

RegionError(1, FilterMonthsYears(sales_month, ["01/2017","02/2017"]))
RegionError(1, NegativeFilterMonthsYears(sales_month, ["01/2017","02/2017"]))

0.0

In [4]:
# compute the total error over the given data
# needed Columns: "NumberOfActualSales" the actual sales, "NumberOfSales" the predicted sales.
def TotalError(data):
    err = 0
    regions = set(data["Region"].tolist())
    for region in regions:
        err += RegionError(region, data)
    return err / len(regions)

#test should return 0 being NumberOfActualSales = NumberOfSales
TotalError(FilterMonthsYears(sales_month, ["01/2017","02/2017"]))

0.0

In [5]:
len(set(sales["StoreID"].tolist()))
sales.shape

(523021, 38)